In [4]:
import simpy
import numpy as np


class Patient:
    def __init__(self, id, env, args):
        self.id = id
        self.env = env
        self.args = args
        self.type = np.random.choice(['Acute Stroke', 'Transient Ischaemic Attack', 'Complex Neurological'],
                                      p=[0.3, 0.4, 0.3])
        self.admission_time = None
        self.length_of_stay = None

    def treatment(self, beds):
        self.admission_time = self.env.now
        with beds.request() as req:
            yield req
            self.length_of_stay = np.random.lognormal(*self.args[self.type]['LOS']).item()
            yield self.env.timeout(self.length_of_stay)
            

class AcuteStrokeUnit:
    def __init__(self, args):
        self.env = simpy.Environment()
        self.args = args
        self.beds = simpy.Resource(self.env, capacity=self.args['Bed Capacity'])
        self.patient_count = 0
        self.patients = []
        self.performance_measure_1 = []
        self.performance_measure_2 = []
        
    def arrivals_generator(self):
        while True:
            for i, t in enumerate(['Acute Stroke', 'Transient Ischaemic Attack', 'Complex Neurological']):
                inter_arrival_time = np.random.exponential(1/self.args[t]['Mean Arrival'])
                self.patient_count += 1
                yield self.env.timeout(inter_arrival_time)
                trace(f'Patient № {self.patient_count} ({t[:3]}) arrives at: {self.env.now:.3f}')
                new_patient = Patient(self.patient_count, self.env, self.args)
                self.patients.append(new_patient)
                self.env.process(new_patient.treatment(self.beds))
                
    def run(self, sim_time):
        self.env.process(self.arrivals_generator())
        self.env.run(until=sim_time)
        
    def calculate_performance_measures(self):
        # Performance Measure 1: Percentage of patients admitted within 4 hours
        total_arrivals = len(self.patients)
        admitted_within_4h = 0
        for p in self.patients:
            if p.admission_time - sim_start_time <= 4 * 24 and p.length_of_stay is not None:
                admitted_within_4h += 1
        pm1 = admitted_within_4h / total_arrivals
        
        # Performance Measure 2: Bed utilization rate
        total_bed_days = sum(p.length_of_stay for p in self.patients)
        bed_utilization_rate = total_bed_days / (len(self.beds) * (sim_end_time - sim_start_time))
        pm2 = bed_utilization_rate
        
        return pm1, pm2


# Model Parameters
args = {
    'Acute Stroke': {
        'Mean Arrival': 1.2,
        'LOS': (np.log(7.4), np.log(8.5))
    },
    'Transient Ischaemic Attack': {
        'Mean Arrival': 9.5,
        'LOS': (np.log(1.8), np.log(2.3))
    },
    'Complex Neurological': {
        'Mean Arrival': 3.5,
        'LOS': (np.log(2.0), np.log(2.5))
    },
    'Bed Capacity': 9
}

# Simulation Parameters
sim_start_time = 0
sim_end_time = 365 * 24 # in hours



env = simpy.Environment()

# Run the simulation
model = AcuteStrokeUnit.run(sim_end_time)

TypeError: run() missing 1 required positional argument: 'sim_time'